## 🧠 Copilot Prompt

Generate a GitHub Actions workflow that automatically runs this notebook using `jupyter nbconvert --execute` and exports the HTML to `docs/validation.html`.

Ensure that `mpmath`, `sympy`, and `numpy` are installed beforehand.

If execution fails, output a warning with traceback.

## Optimization Notes

This notebook has been optimized to prevent GitHub Actions timeout by:
- Reducing precision from 50 to 25 decimal places
- Using precomputed zeros instead of mp.zetazero()
- Reducing computational parameters (N, P, K)
- Adding execution time monitoring

In [ ]:
# Import libraries with optimized precision
import mpmath as mp
import sympy as sp
import numpy as np
import time
import os

# Set precision (reduced from 50 to 25 for faster execution)
mp.mp.dps = 25

# Load precomputed zeros if available
zeros_data = []
try:
    zeros_file = 'zeros/zeros_t1e8.txt'
    if not os.path.exists(zeros_file):
        # Try relative path from notebooks directory
        zeros_file = '../zeros/zeros_t1e8.txt'
    
    with open(zeros_file, 'r') as f:
        for line in f:
            zeros_data.append(mp.mpf(line.strip()))
    print(f"✅ Loaded {len(zeros_data)} precomputed zeros from {zeros_file}")
except FileNotFoundError:
    print("⚠️ Warning: Precomputed zeros not found. Will compute a smaller subset.")
    # Fallback: compute small subset of zeros
    for n in range(1, 101):  # Only 100 zeros as fallback
        zeros_data.append(mp.im(mp.zetazero(n)))
    print(f"✅ Computed {len(zeros_data)} zeros as fallback")

# Log start time
start_time = time.time()
print(f"🚀 Starting validation at {time.ctime(start_time)}")
print(f"📊 Precision: {mp.mp.dps} decimal places")

In [ ]:
# Define test functions (unchanged for mathematical accuracy)
def f1(u): return mp.exp(-u**2/2) if abs(u) <= 3 else mp.mpf(0)
def f2(u): return mp.exp(-u**2) if abs(u) <= 2 else mp.mpf(0)
def f3(u): return (1 - u**2/4)**2 if abs(u) <= 2 else mp.mpf(0)

def fhat(f, s, lim):
    """Fourier transform - reduced maxdegree for speed"""
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=8)  # Reduced from 10

print("✅ Test functions defined")

In [ ]:
def prime_sum(f, P=100, K=5):  # Reduced even further for CI
    """Prime sum - reduced parameters for speed"""
    s = mp.mpf(0)
    primes = list(sp.primerange(2, 7919))[:P]
    for p in primes:
        lp = mp.log(p)
        for k in range(1, K+1):
            s += lp * f(k * lp)
    print(f"  Prime sum computed with P={P}, K={K} ({P*K} terms)")
    return s

def A_infty(f, sigma0=2.0, lim=3, T=10):  # Reduced T even further for CI
    """Archimedean contribution - reduced integration range"""
    def integrand(t):
        s = mp.mpc(sigma0, t)
        return (mp.digamma(s/2) - mp.log(mp.pi)) * fhat(f, s, lim)
    integ = mp.quad(integrand, [-T, T], maxdegree=8) / (2 * mp.pi)  # Reduced maxdegree
    res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
    print(f"  Archimedean sum computed with T={T}")
    return integ - res1

def zero_sum_precomputed(f, zeros, N=100, lim=3):  # Reduced even further for CI
    """Zero sum using precomputed zeros - much faster than mp.zetazero"""
    s = mp.mpf(0)
    N_actual = min(N, len(zeros))
    for n in range(N_actual):
        gamma = zeros[n]
        s += fhat(f, 1j * gamma, lim).real
    print(f"  Zero sum computed with {N_actual} precomputed zeros")
    return s

def zero_sum_fallback(f, N=200, lim=3):  # Much smaller fallback
    """Fallback zero sum using mp.zetazero - for when precomputed zeros unavailable"""
    s = mp.mpf(0)
    print(f"  Computing {N} zeros using mp.zetazero (slower)...")
    for n in range(1, N+1):
        rho = mp.zetazero(n)
        s += fhat(f, mp.im(rho), lim).real
        if n % 50 == 0:  # Progress indicator
            print(f"    Progress: {n}/{N} zeros computed")
    return s

print("✅ Computation functions defined with optimized parameters")

In [ ]:
# Main validation computation with timing
test_cases = [('f1', f1, 3), ('f2', f2, 2), ('f3', f3, 2)]
results = []

for i, (fname, f, lim) in enumerate(test_cases):
    print(f"\n🔬 Computing {fname} ({i+1}/{len(test_cases)})...")
    case_start = time.time()
    
    # Prime sum
    print("  Computing prime sum...")
    ps = prime_sum(f)
    
    # Archimedean contribution
    print("  Computing Archimedean contribution...")
    ain = A_infty(f, lim=lim)
    
    # Zero sum - use precomputed if available
    print("  Computing zero sum...")
    if zeros_data:
        zs = zero_sum_precomputed(f, zeros_data, lim=lim)
    else:
        zs = zero_sum_fallback(f, lim=lim)
    
    # Calculate results
    tot = ps + ain
    err = abs(tot - zs)
    case_time = time.time() - case_start
    
    result = {
        'function': fname,
        'prime_archimedean': float(tot.real),
        'zero_side': float(zs.real),
        'absolute_error': float(err),
        'relative_error': float(err / abs(tot)) if abs(tot) > 0 else float('inf'),
        'computation_time': case_time
    }
    results.append(result)
    
    print(f"  ✅ {fname}: prime+A={float(tot.real):.6f}, zero={float(zs.real):.6f}, err={float(err):.2e}, time={case_time:.1f}s")

total_time = time.time() - start_time
print(f"\n🎉 All computations completed in {total_time:.1f} seconds")

In [ ]:
# Summary and results table
print("\n📊 VALIDATION RESULTS SUMMARY")
print("=" * 60)
print(f"{'Function':<8} {'Prime+Arch':<12} {'Zero Side':<12} {'Error':<10} {'Time(s)':<8}")
print("-" * 60)

for r in results:
    print(f"{r['function']:<8} {r['prime_archimedean']:<12.6f} {r['zero_side']:<12.6f} {r['absolute_error']:<10.2e} {r['computation_time']:<8.1f}")

print("-" * 60)
print(f"Total execution time: {total_time:.1f} seconds")
print(f"Precision used: {mp.mp.dps} decimal places")
print(f"Zeros source: {'Precomputed file' if zeros_data else 'mp.zetazero fallback'}")

# Check if all errors are reasonable
max_error = max(r['absolute_error'] for r in results)
print(f"\nMaximum absolute error: {max_error:.2e}")
if max_error < 1e-4:
    print("✅ All validations PASSED (errors < 1e-4)")
elif max_error < 1e-2:
    print("⚠️  Validations acceptable but with reduced precision")
else:
    print("❌ Large errors detected - check computation parameters")
    
print(f"\n🕐 Completed at {time.ctime()}")